Questo notebook per Colaboratory di Google mostra come identificare la lingua di un testo e tradurla nella lingua desiderata utilizzando i modelli Facebook fasttext e No Language Left Behind (NLLB)

In [ ]:
!pip install fasttext

!wget https://dl.fbaipublicfiles.com/nllb/lid/lid218e.bin
# !wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

Iniziamo a rilevare la lingua dell'input di testo

In [ ]:
testo = input("Inserisci il testo da riconoscere: ")

In [ ]:
import fasttext

plm = "/content/lid218e.bin"
model = fasttext.load_model(plm)
p = model.predict(testo, k=-1, threshold=0.5)
input_lang = p[0][0].replace('__label__', '')
print(input_lang)

Dopodiché, prendiamo il testo inserito e l'etichetta prevista e li diamo in pasto a NLLB, che traduce il testo dalla lingua originale alla lingua supportata da NLLB

In [ ]:
!pip install transformers

In [ ]:
model_name = 'facebook/nllb-200-distilled-600M'
# model_name = 'facebook/nllb-200-1.3B'
# model_name = 'facebook/nllb-200-3.3B'
# model_name = 'facebook/nllb-200-distilled-1.3B'

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

if not torch.cuda.is_available():
  print('WARNING: You may want to change the runtime to GPU for faster training!')
  device = 'cpu'
else:
  device = 'cuda:0'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)
output_lang = 'eng_Latn'
translation_pipeline = pipeline('translation',
                                model=model,
                                tokenizer=tokenizer,
                                src_lang=input_lang,
                                tgt_lang=output_lang,
                                max_length = 400,
                                device=device)
output = translation_pipeline(testo)
print(output[0]['translation_text'])